In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pylab as plt
import sys, gc, warnings, random, math, time, datetime 
from tqdm import tqdm
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')

from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from scipy.stats.stats import pearsonr

import xgboost as xgb
import lightgbm as lgb

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from utils import *

import os
print(os.listdir("../input/features/"))

['train_y.pkl', 'train_uid_features.pkl', 'train_features.pkl', 'useful_features.npy', 'test_features.pkl', 'test_uid_features.pkl', 'train_basic_features.pkl', 'rm_features.npy', 'test.pkl', 'test_basic_features.pkl', 'train.pkl']


In [2]:
train = pd.read_pickle('../input/features/train_basic_features.pkl')
test = pd.read_pickle('../input/features/test_basic_features.pkl')
train_y = pd.read_pickle('../input/features/train_y.pkl')

rm_features = np.load('../input/features/rm_features.npy', allow_pickle=True)
rm_features = list(rm_features)

print(train.shape)
print(train_y.shape)
print(test.shape)
print(len(rm_features))

(590540, 229)
(590540,)
(506691, 229)
11


In [3]:
train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,DT,DT_month,DT_week_year,DT_day_year,DT_hour,DT_day_week,DT_day,DT_hour_fq_enc,DT_day_week_fq_enc,DT_day_fq_enc,card1_fq_enc,card2_fq_enc,card3_fq_enc,card5_fq_enc,addr1_fq_enc,addr2_fq_enc,email_check,P_emaildomain_fq_enc,R_emaildomain_fq_enc,D1_day_mean,DT_day_year_prdt,D1_prdt_day_mean,D1_week_mean,DT_week_year_prdt,D1_prdt_week_mean,D2_day_mean,D2_prdt_day_mean,D2_week_mean,D2_prdt_week_mean,D3_day_mean,D3_prdt_day_mean,D3_week_mean,D3_prdt_week_mean,D4_day_mean,D4_prdt_day_mean,D4_week_mean,D4_prdt_week_mean,D5_day_mean,D5_prdt_day_mean,D5_week_mean,D5_prdt_week_mean,D6_day_mean,D6_prdt_day_mean,D6_week_mean,D6_prdt_week_mean,D7_day_mean,D7_prdt_day_mean,D7_week_mean,D7_prdt_week_mean,D10_day_mean,D10_prdt_day_mean,D10_week_mean,D10_prdt_week_mean,D11_day_mean,D11_prdt_day_mean,D11_week_mean,D11_prdt_week_mean,D12_day_mean,D12_prdt_day_mean,D12_week_mean,D12_prdt_week_mean,D13_day_mean,D13_prdt_day_mean,D13_week_mean,D13_prdt_week_mean,D14_day_mean,D14_prdt_day_mean,D14_week_mean,D14_prdt_week_mean,D15_day_mean,D15_prdt_day_mean,D15_week_mean,D15_prdt_week_mean,M_sum,M_na,V1_pca_0,V1_pca_1,V1_pca_2,V1_pca_3,V1_pca_4,V12_pca_0,V12_pca_1,V12_pca_2,V12_pca_3,V12_pca_4,V35_pca_0,V35_pca_1,V35_pca_2,V35_pca_3,V35_pca_4,V53_pca_0,V53_pca_1,V53_pca_2,V53_pca_3,V53_pca_4,V75_pca_0,V75_pca_1,V75_pca_2,V75_pca_3,V75_pca_4,V95_pca_0,V95_pca_1,V95_pca_2,V95_pca_3,V95_pca_4,V138_pca_0,V138_pca_1,V138_pca_2,V138_pca_3,V138_pca_4,V167_pca_0,V167_pca_1,V167_pca_2,V167_pca_3,V167_pca_4,V217_pca_0,V217_pca_1,V217_pca_2,V217_pca_3,V217_pca_4,V279_pca_0,V279_pca_1,V279_pca_2,V279_pca_3,V279_pca_4,V322_pca_0,V322_pca_1,V322_pca_2,V322_pca_3,V322_pca_4,id_33_0,id_33_1,id_30_device,id_30_version,lastest_browser
TransactionID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400,68.5,0.729707,13926.0,NaN,150.0,0.008680,142.0,0.243930,315.0,87.0,19.0,NaN,49,49,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,1.0,1.0,1.0,0.112052,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown_device,NaN,NaN,460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017-12-01 00:00:00,12,48,335,0,4,1,0.062342,0.163816,0.036737,0.000051,0.016029,0.872054,0.000282,0.039221,0.871662,0,0.149146,0.751045,-88.75,335_W,-106.937500,-82.4375,48_W,-104.750000,NaN,NaN,NaN,NaN,-15.546875,-14.6875,-14.84375,-13.84375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-120.5,-134.25,-114.8125,-130.625,-142.125,-142.125,-148.5,-148.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-195.75,-209.875,-199.375,-214.75,4,3,1.105709,-0.672874,0.09168,-0.041502,-0.112852,-0.517778,0.343313,-0.795382,-0.072634,-0.067785,-2.189012,0.570393,0.716233,-0.092708,0.086531,-0.443217,0.421562,-0.746061,-0.258886,-0.010403,-0.379178,0.513620,0.799062,-0.659385,0.077510,-252.980928,-96.751761,48.857959,-71.865657,68.485602,-10376.501283,-105.779936,20.137186,-23.409744,1.187370,-299.047397,-24.895938,-34.844658,-1.362068,13.039909,-117.757307,-14.162958,-9.824124,-9.723595,4.034396,-330.417064,-99.633832,10.402851,-4.653791,-134.096963,-186.381735,-8.941901,11.315498,-2.795801,-5.815186,0,0,7,0,0.0
2987001,0,86401,29.0,0.729707,2755.0,404.0,150.0,0.316602,1

In [4]:
for df in [train, test]:
    for col in ['D1', 'D2','D3', 'D4', 'D5', 'D6','D7','D10','D11','D12','D13','D14','D15']:
        df[col] = df[col] - train['DT_day_year'] + train['DT_day_year'].min()
        df[col] = df[col].apply(lambda y: max(0,y))
#         temp_df = pd.concat([train[[col]], test[[col]]])
#         fq_encode = temp_df[col].value_counts(dropna=False) / len(temp_df)
#         fq_encode = fq_encode.to_dict()
#         df[col+'_fq_enc'] = df[col].map(fq_encode)


In [5]:
# fe with amount (user id)
def fe_uid(train_df, test_df, rm_features):
    
    print('==> processing uid...')
    
    # user id, save for later aggregation
    for df in [train_df, test_df]:
        df['cents'] = df['TransactionAmt'].apply(lambda x: x - int(x))
        df['uid0'] = df['cents'].astype(str)+'_'+df['ProductCD'].astype(str)
        
        ### universal user
        # basic: uid2(card), uid3(addr), uid5(email)
        # aggr on uid4, uid5, uid7, uid8(*)
        # card
        df['uid1'] = df['card1'].astype(str)+'_'+df['card2'].astype(str)
        df['uid2'] = df['uid1'] + df['card3'].astype(str)+'_'+df['card5'].astype(str) # card holder
        
        # addr
        df['uid3'] =  df['addr1'].astype(str)+'_'+df['addr2'].astype(str) # family addr
        df['uid4'] =  df['uid2'].astype(str)+'_'+df['uid3'].astype(str)+'_'+df['ProductCD'].astype(str) # card holder with addr
                
        # email
        df['uid5'] = df['P_emaildomain'].astype(str)+'_'+df['R_emaildomain'].astype(str)
        df['uid6'] = df['uid2'].astype(str)+'_'+df['uid5'].astype(str)+'_'+df['ProductCD'].astype(str)
        df['uid7'] = df['uid3'].astype(str)+'_'+df['uid5'].astype(str)+'_'+df['ProductCD'].astype(str)
        
        # card + addr + email => specific user
        df['uid8'] = df['uid2'].astype(str)+'_'+df['uid3'].astype(str)+'_'+df['uid5'].astype(str)+'_'+df['ProductCD'].astype(str)

        ### uid4
        df['uid9'] = df['uid4'].astype(str)+'_'+df['M1'].astype(str)+'_'+df['M2'].astype(str)+'_'+df['M3'].astype(str)
        df['uid10'] = df['uid4'].astype(str)+'_'+df['M4'].astype(str)+'_'+df['M5'].astype(str)+'_'+df['M6'].astype(str)
        df['uid11'] = df['uid4'].astype(str)+'_'+df['M7'].astype(str)+'_'+df['M8'].astype(str)+'_'+df['M9'].astype(str)
        
        df['uid12'] = df['uid4'].astype(str)+'_'+df['C1'].astype(str)+'_'+df['C2'].astype(str)
        df['uid13'] = df['uid4'].astype(str)+'_'+df['C5'].astype(str)+'_'+df['C6'].astype(str)
        df['uid14'] = df['uid4'].astype(str)+'_'+df['C4'].astype(str)+'_'+df['C6'].astype(str)+'_'+df['C8'].astype(str)+'_'+df['C10'].astype(str)
        df['uid15'] = df['uid4'].astype(str)+'_'+df['C9'].astype(str)+'_'+df['C11'].astype(str)+'_'+df['C12'].astype(str)
        
        df['uid16'] = df['uid4'].astype(str)+'_'+df['D1'].astype(str)+'_'+df['D2'].astype(str)
        df['uid17'] = df['uid4'].astype(str)+'_'+df['D3'].astype(str)+'_'+df['D4'].astype(str)+'_'+df['D5'].astype(str)
        df['uid18'] = df['uid4'].astype(str)+'_'+df['D4'].astype(str)+'_'+df['D6'].astype(str)
        
        df['uid19'] = df['uid4'].astype(str)+'_'+df['D8'].astype(str)+'_'+df['D9'].astype(str)
        
        df['uid20'] = df['uid4'].astype(str)+'_'+df['D10'].astype(str)+'_'+df['D11'].astype(str)+'_'+df['D15'].astype(str)
        df['uid21'] = df['uid4'].astype(str)+'_'+df['D12'].astype(str)+'_'+df['D13'].astype(str)+'_'+df['D14'].astype(str)
        
        df['uid22'] = df['uid4'].astype(str)+'_'+df['dist1'].astype(str)+'_'+df['dist2'].astype(str)
        
        df['uid23'] = df['uid4'].astype(str)+'_'+df['id_01'].astype(str)+'_'+df['id_03'].astype(str)+'_'+df['id_04'].astype(str)
        df['uid24'] = df['uid4'].astype(str)+'_'+df['id_05'].astype(str)+'_'+df['id_06'].astype(str)+'_'+df['id_07'].astype(str)+'_'+df['id_08'].astype(str)
        df['uid25'] = df['uid4'].astype(str)+'_'+df['id_19'].astype(str)+'_'+df['id_20'].astype(str)
        df['uid26'] = df['uid4'].astype(str)+'_'+df['id_21'].astype(str)+'_'+df['id_22'].astype(str)+'_'+df['id_23'].astype(str)+'_'+df['id_24'].astype(str)+'_'+df['id_25'].astype(str)+'_'+df['id_26'].astype(str)+'_'+df['id_27'].astype(str) 
        df['uid27'] = df['uid4'].astype(str)+'_'+df['id_30'].astype(str)+'_'+df['id_31'].astype(str)+'_'+df['id_32'].astype(str)+'_'+df['id_33'].astype(str)+'_'+df['id_34'].astype(str)+'_'+df['id_35'].astype(str)+'_'+df['id_36'].astype(str) +df['id_37'].astype(str)+'_'+df['id_38'].astype(str)+'_'+df['DeviceInfo'].astype(str)+'_'+df['DeviceType'].astype(str)
        
        df['uid28'] = df['uid4'].astype(str)+'_'+df['TransactionAmt'].astype(str)
        
        df['uid29'] = df['uid4'].astype(str)+'_'+df['DT_week_year'].astype(str)
        df['uid30'] = df['uid4'].astype(str)+'_'+df['DT_day_year'].astype(str)
        df['uid31'] = df['uid4'].astype(str)+'_'+df['DT_day_week'].astype(str)
        df['uid32'] = df['uid4'].astype(str)+'_'+df['DT_day'].astype(str)
        df['uid33'] = df['uid4'].astype(str)+'_'+df['DT_hour'].astype(str)
        
        ### uid5
        df['uid34'] = df['uid5'].astype(str)+'_'+df['M1'].astype(str)+'_'+df['M2'].astype(str)+'_'+df['M3'].astype(str)
        df['uid35'] = df['uid5'].astype(str)+'_'+df['M4'].astype(str)+'_'+df['M5'].astype(str)+'_'+df['M6'].astype(str)
        df['uid36'] = df['uid5'].astype(str)+'_'+df['M7'].astype(str)+'_'+df['M8'].astype(str)+'_'+df['M9'].astype(str)
        
        df['uid37'] = df['uid5'].astype(str)+'_'+df['C1'].astype(str)+'_'+df['C2'].astype(str)
        df['uid38'] = df['uid5'].astype(str)+'_'+df['C5'].astype(str)+'_'+df['C6'].astype(str)
        df['uid39'] = df['uid5'].astype(str)+'_'+df['C4'].astype(str)+'_'+df['C6'].astype(str)+'_'+df['C8'].astype(str)+'_'+df['C10'].astype(str)
        df['uid40'] = df['uid5'].astype(str)+'_'+df['C9'].astype(str)+'_'+df['C11'].astype(str)+'_'+df['C12'].astype(str)
        
        df['uid41'] = df['uid5'].astype(str)+'_'+df['D1'].astype(str)+'_'+df['D2'].astype(str)
        df['uid42'] = df['uid5'].astype(str)+'_'+df['D3'].astype(str)+'_'+df['D4'].astype(str)+'_'+df['D5'].astype(str)
        df['uid43'] = df['uid5'].astype(str)+'_'+df['D4'].astype(str)+'_'+df['D6'].astype(str)
        
        df['uid44'] = df['uid5'].astype(str)+'_'+df['D8'].astype(str)+'_'+df['D9'].astype(str)
        
        df['uid45'] = df['uid5'].astype(str)+'_'+df['D10'].astype(str)+'_'+df['D11'].astype(str)+'_'+df['D15'].astype(str)
        df['uid46'] = df['uid5'].astype(str)+'_'+df['D12'].astype(str)+'_'+df['D13'].astype(str)+'_'+df['D14'].astype(str)
        
        df['uid47'] = df['uid5'].astype(str)+'_'+df['dist1'].astype(str)+'_'+df['dist2'].astype(str)
        
        df['uid48'] = df['uid5'].astype(str)+'_'+df['id_01'].astype(str)+'_'+df['id_03'].astype(str)+'_'+df['id_04'].astype(str)
        df['uid49'] = df['uid5'].astype(str)+'_'+df['id_05'].astype(str)+'_'+df['id_06'].astype(str)+'_'+df['id_07'].astype(str)+'_'+df['id_08'].astype(str)
        df['uid50'] = df['uid5'].astype(str)+'_'+df['id_19'].astype(str)+'_'+df['id_20'].astype(str)
        df['uid51'] = df['uid5'].astype(str)+'_'+df['id_21'].astype(str)+'_'+df['id_22'].astype(str)+'_'+df['id_23'].astype(str)+'_'+df['id_24'].astype(str)+'_'+df['id_25'].astype(str)+'_'+df['id_26'].astype(str)+'_'+df['id_27'].astype(str) 
        df['uid52'] = df['uid5'].astype(str)+'_'+df['id_30'].astype(str)+'_'+df['id_31'].astype(str)+'_'+df['id_32'].astype(str)+'_'+df['id_33'].astype(str)+'_'+df['id_34'].astype(str)+'_'+df['id_35'].astype(str)+'_'+df['id_36'].astype(str) +df['id_37'].astype(str)+'_'+df['id_38'].astype(str)+'_'+df['DeviceInfo'].astype(str)+'_'+df['DeviceType'].astype(str)
        
        df['uid53'] = df['uid5'].astype(str)+'_'+df['TransactionAmt'].astype(str)
        
        df['uid54'] = df['uid5'].astype(str)+'_'+df['DT_week_year'].astype(str)
        df['uid55'] = df['uid5'].astype(str)+'_'+df['DT_day_year'].astype(str)
        df['uid56'] = df['uid5'].astype(str)+'_'+df['DT_day_week'].astype(str)
        df['uid57'] = df['uid5'].astype(str)+'_'+df['DT_day'].astype(str)
        df['uid58'] = df['uid5'].astype(str)+'_'+df['DT_hour'].astype(str)
        
        ### uid7
        df['uid59'] = df['uid7'].astype(str)+'_'+df['M1'].astype(str)+'_'+df['M2'].astype(str)+'_'+df['M3'].astype(str)
        df['uid60'] = df['uid7'].astype(str)+'_'+df['M4'].astype(str)+'_'+df['M5'].astype(str)+'_'+df['M6'].astype(str)
        df['uid61'] = df['uid7'].astype(str)+'_'+df['M7'].astype(str)+'_'+df['M8'].astype(str)+'_'+df['M9'].astype(str)
        
        df['uid62'] = df['uid7'].astype(str)+'_'+df['C1'].astype(str)+'_'+df['C2'].astype(str)
        df['uid63'] = df['uid7'].astype(str)+'_'+df['C5'].astype(str)+'_'+df['C6'].astype(str)
        df['uid64'] = df['uid7'].astype(str)+'_'+df['C4'].astype(str)+'_'+df['C6'].astype(str)+'_'+df['C8'].astype(str)+'_'+df['C10'].astype(str)
        df['uid65'] = df['uid7'].astype(str)+'_'+df['C9'].astype(str)+'_'+df['C11'].astype(str)+'_'+df['C12'].astype(str)
        
        df['uid66'] = df['uid7'].astype(str)+'_'+df['D1'].astype(str)+'_'+df['D2'].astype(str)
        df['uid67'] = df['uid7'].astype(str)+'_'+df['D3'].astype(str)+'_'+df['D4'].astype(str)+'_'+df['D5'].astype(str)
        df['uid68'] = df['uid7'].astype(str)+'_'+df['D4'].astype(str)+'_'+df['D6'].astype(str)
        
        df['uid69'] = df['uid7'].astype(str)+'_'+df['D8'].astype(str)+'_'+df['D9'].astype(str)
        
        df['uid70'] = df['uid7'].astype(str)+'_'+df['D10'].astype(str)+'_'+df['D11'].astype(str)+'_'+df['D15'].astype(str)
        df['uid71'] = df['uid7'].astype(str)+'_'+df['D12'].astype(str)+'_'+df['D13'].astype(str)+'_'+df['D14'].astype(str)
        
        df['uid72'] = df['uid7'].astype(str)+'_'+df['dist1'].astype(str)+'_'+df['dist2'].astype(str)
        
        df['uid73'] = df['uid7'].astype(str)+'_'+df['id_01'].astype(str)+'_'+df['id_03'].astype(str)+'_'+df['id_04'].astype(str)
        df['uid74'] = df['uid7'].astype(str)+'_'+df['id_05'].astype(str)+'_'+df['id_06'].astype(str)+'_'+df['id_07'].astype(str)+'_'+df['id_08'].astype(str)
        df['uid75'] = df['uid7'].astype(str)+'_'+df['id_19'].astype(str)+'_'+df['id_20'].astype(str)
        df['uid76'] = df['uid7'].astype(str)+'_'+df['id_21'].astype(str)+'_'+df['id_22'].astype(str)+'_'+df['id_23'].astype(str)+'_'+df['id_24'].astype(str)+'_'+df['id_25'].astype(str)+'_'+df['id_26'].astype(str)+'_'+df['id_27'].astype(str) 
        df['uid77'] = df['uid7'].astype(str)+'_'+df['id_30'].astype(str)+'_'+df['id_31'].astype(str)+'_'+df['id_32'].astype(str)+'_'+df['id_33'].astype(str)+'_'+df['id_34'].astype(str)+'_'+df['id_35'].astype(str)+'_'+df['id_36'].astype(str) +df['id_37'].astype(str)+'_'+df['id_38'].astype(str)+'_'+df['DeviceInfo'].astype(str)+'_'+df['DeviceType'].astype(str)
        
        df['uid78'] = df['uid7'].astype(str)+'_'+df['TransactionAmt'].astype(str)
        
        df['uid79'] = df['uid7'].astype(str)+'_'+df['DT_week_year'].astype(str)
        df['uid80'] = df['uid7'].astype(str)+'_'+df['DT_day_year'].astype(str)
        df['uid81'] = df['uid7'].astype(str)+'_'+df['DT_day_week'].astype(str)
        df['uid82'] = df['uid7'].astype(str)+'_'+df['DT_day'].astype(str)
        df['uid83'] = df['uid7'].astype(str)+'_'+df['DT_hour'].astype(str)
        
        ### uid8
        df['uid84'] = df['uid8'].astype(str)+'_'+df['M1'].astype(str)+'_'+df['M2'].astype(str)+'_'+df['M3'].astype(str)
        df['uid85'] = df['uid8'].astype(str)+'_'+df['M4'].astype(str)+'_'+df['M5'].astype(str)+'_'+df['M6'].astype(str)
        df['uid86'] = df['uid8'].astype(str)+'_'+df['M7'].astype(str)+'_'+df['M8'].astype(str)+'_'+df['M9'].astype(str)
        
        df['uid87'] = df['uid8'].astype(str)+'_'+df['C1'].astype(str)+'_'+df['C2'].astype(str)
        df['uid88'] = df['uid8'].astype(str)+'_'+df['C5'].astype(str)+'_'+df['C6'].astype(str)
        df['uid89'] = df['uid8'].astype(str)+'_'+df['C4'].astype(str)+'_'+df['C6'].astype(str)+'_'+df['C8'].astype(str)+'_'+df['C10'].astype(str)
        df['uid90'] = df['uid8'].astype(str)+'_'+df['C9'].astype(str)+'_'+df['C11'].astype(str)+'_'+df['C12'].astype(str)
        
        df['uid91'] = df['uid8'].astype(str)+'_'+df['D1'].astype(str)+'_'+df['D2'].astype(str)
        df['uid92'] = df['uid8'].astype(str)+'_'+df['D3'].astype(str)+'_'+df['D4'].astype(str)+'_'+df['D5'].astype(str)
        df['uid93'] = df['uid8'].astype(str)+'_'+df['D4'].astype(str)+'_'+df['D6'].astype(str)
        
        df['uid94'] = df['uid8'].astype(str)+'_'+df['D8'].astype(str)+'_'+df['D9'].astype(str)
        
        df['uid95'] = df['uid8'].astype(str)+'_'+df['D10'].astype(str)+'_'+df['D11'].astype(str)+'_'+df['D15'].astype(str)
        df['uid96'] = df['uid8'].astype(str)+'_'+df['D12'].astype(str)+'_'+df['D13'].astype(str)+'_'+df['D14'].astype(str)
        
        df['uid97'] = df['uid8'].astype(str)+'_'+df['dist1'].astype(str)+'_'+df['dist2'].astype(str)
        
        df['uid98'] = df['uid8'].astype(str)+'_'+df['id_01'].astype(str)+'_'+df['id_03'].astype(str)+'_'+df['id_04'].astype(str)
        df['uid99'] = df['uid8'].astype(str)+'_'+df['id_05'].astype(str)+'_'+df['id_06'].astype(str)+'_'+df['id_07'].astype(str)+'_'+df['id_08'].astype(str)
        df['uid100'] = df['uid8'].astype(str)+'_'+df['id_19'].astype(str)+'_'+df['id_20'].astype(str)
        df['uid101'] = df['uid8'].astype(str)+'_'+df['id_21'].astype(str)+'_'+df['id_22'].astype(str)+'_'+df['id_23'].astype(str)+'_'+df['id_24'].astype(str)+'_'+df['id_25'].astype(str)+'_'+df['id_26'].astype(str)+'_'+df['id_27'].astype(str) 
        df['uid102'] = df['uid8'].astype(str)+'_'+df['id_30'].astype(str)+'_'+df['id_31'].astype(str)+'_'+df['id_32'].astype(str)+'_'+df['id_33'].astype(str)+'_'+df['id_34'].astype(str)+'_'+df['id_35'].astype(str)+'_'+df['id_36'].astype(str) +df['id_37'].astype(str)+'_'+df['id_38'].astype(str)+'_'+df['DeviceInfo'].astype(str)+'_'+df['DeviceType'].astype(str)
        
        df['uid103'] = df['uid8'].astype(str)+'_'+df['TransactionAmt'].astype(str)
        
        df['uid104'] = df['uid8'].astype(str)+'_'+df['DT_week_year'].astype(str)
        df['uid105'] = df['uid8'].astype(str)+'_'+df['DT_day_year'].astype(str)
        df['uid106'] = df['uid8'].astype(str)+'_'+df['DT_day_week'].astype(str)
        df['uid107'] = df['uid8'].astype(str)+'_'+df['DT_day'].astype(str)
        df['uid108'] = df['uid8'].astype(str)+'_'+df['DT_hour'].astype(str)
        
        
    uid_list = ['uid' + str(i) for i in range(109)]
    cid_list = [] #['cid' + str(i) for i in range(1, 16)]
            
    tmp_rm = [] + uid_list + cid_list

    rm_features = rm_features + tmp_rm
    return train_df, test_df, rm_features, uid_list, cid_list
# train, test, rm_features = fe_uid(train, test, rm_features)

In [6]:
# group aggregation
def fe_agg(train_df, test_df, rm_features, uid_list, cid_list):

    print('==> processing aggregation...')

    uid_cols = ['card1', 'card2', 'card3', 'card5'] + uid_list
    
    for col in tqdm_notebook(uid_cols):
        # aggr: mean, std, min, max, sum
        for agg_type in ['mean', 'std']:
            for agg_col in ['TransactionAmt']:
                new_col_name = col + '_' + agg_col + '_' + agg_type
                temp_df = pd.concat(
                    [train_df[[col, agg_col]], test_df[[col, agg_col]]])
                temp_df = temp_df.groupby([col])[agg_col].agg([
                    agg_type
                ]).reset_index().rename(columns={agg_type: new_col_name})

                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_col_name].to_dict()

                train_df[new_col_name] = train_df[col].map(temp_df)
                test_df[new_col_name] = test_df[col].map(temp_df)  
                
    for col in tqdm_notebook(uid_cols):
        # aggr: value - mean
        for agg_type in ['mean']:
            for agg_col in ['TransactionAmt']:
                new_col_name = col + '_' + agg_col + '_' + agg_type + '_diff'
                temp_df = pd.concat(
                    [train_df[[col, agg_col]], test_df[[col, agg_col]]])
                temp_df = temp_df.groupby([col])[agg_col].agg([
                    agg_type
                ]).reset_index().rename(columns={agg_type: new_col_name})

                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_col_name].to_dict()

                train_df[new_col_name] = train_df[col].map(temp_df)
                test_df[new_col_name] = test_df[col].map(temp_df) 
                train_df[new_col_name] = train_df[agg_col] - train_df[new_col_name]
                test_df[new_col_name] = test_df[agg_col] - test_df[new_col_name]
                
    for col in tqdm_notebook(uid_cols + cid_list):
        # count
        for agg_type in ['count']:
            for agg_col in ['TransactionDT']:
                new_col_name = col + '_' + agg_type
                temp_df = pd.concat(
                    [train_df[[col, agg_col]], test_df[[col, agg_col]]])
                temp_df = temp_df.groupby([col])[agg_col].agg([
                    agg_type
                ]).reset_index().rename(columns={agg_type: new_col_name})

                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_col_name].to_dict()

                train_df[new_col_name] = train_df[col].map(temp_df)
                test_df[new_col_name] = test_df[col].map(temp_df)
        
        
    train_df = train_df.replace(np.inf, 999)
    test_df = test_df.replace(np.inf, 999)

#     for df in [train_df, test_df]:
#         for col in ['D1', 'D2','D3', 'D4', 'D5', 'D6','D7','D10','D11','D12','D13','D14','D15']:
#             df[col] = df[col] - train['DT_day_year'] + train['DT_day_year'].min()
#             df[col] = df[col].apply(lambda y: max(0,y))
#             temp_df = pd.concat([train_df[[col]], test_df[[col]]])
#             fq_encode = temp_df[col].value_counts(dropna=False) / len(temp_df)
#             fq_encode = fq_encode.to_dict()
#             train_df[col+'_fq_enc'] = train_df[col].map(fq_encode)
#             test_df[col+'_fq_enc']  = test_df[col].map(fq_encode)
    
    tmp_rm = []
    rm_features = rm_features + tmp_rm
    return train_df, test_df, rm_features

# train, test, rm_features = fe_agg(train, test, rm_features)

In [7]:
def fe(train_df, test_df, rm_features):
    
    # fe with amount (user id)
    train_df, test_df, rm_features, uid_list, cid_list = fe_uid(train_df, test_df, rm_features)
    
    # group aggregation
    train_df, test_df, rm_features = fe_agg(train_df, test_df, rm_features, uid_list, cid_list)
    
    return train_df, test_df, rm_features

train, test, rm_features = fe(train, test, rm_features)

==> processing uid...
==> processing aggregation...


In [8]:
print(train.shape)
print(train_y.shape)
print(test.shape)
print(len(rm_features))

train.head()

(590540, 791)
(590540,)
(506691, 791)
120


,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,DT,DT_month,DT_week_year,DT_day_year,DT_hour,DT_day_week,DT_day,DT_hour_fq_enc,DT_day_week_fq_enc,DT_day_fq_enc,card1_fq_enc,card2_fq_enc,card3_fq_enc,card5_fq_enc,addr1_fq_enc,addr2_fq_enc,email_check,P_emaildomain_fq_enc,R_emaildomain_fq_enc,D1_day_mean,DT_day_year_prdt,D1_prdt_day_mean,D1_week_mean,DT_week_year_prdt,D1_prdt_week_mean,D2_day_mean,D2_prdt_day_mean,D2_week_mean,D2_prdt_week_mean,D3_day_mean,D3_prdt_day_mean,D3_week_mean,D3_prdt_week_mean,D4_day_mean,D4_prdt_day_mean,D4_week_mean,D4_prdt_week_mean,D5_day_mean,D5_prdt_day_mean,D5_week_mean,D5_prdt_week_mean,D6_day_mean,D6_prdt_day_mean,D6_week_mean,D6_prdt_week_mean,D7_day_mean,D7_prdt_day_mean,D7_week_mean,D7_prdt_week_mean,D10_day_mean,D10_prdt_day_mean,D10_week_mean,D10_prdt_week_mean,D11_day_mean,D11_prdt_day_mean,D11_week_mean,D11_prdt_week_mean,D12_day_mean,D12_prdt_day_mean,D12_week_mean,D12_prdt_week_mean,D13_day_mean,D13_prdt_day_mean,D13_week_mean,D13_prdt_week_mean,D14_day_mean,D14_prdt_day_mean,D14_week_mean,D14_prdt_week_mean,D15_day_mean,D15_prdt_day_mean,D15_week_mean,D15_prdt_week_mean,M_sum,M_na,V1_pca_0,V1_pca_1,V1_pca_2,V1_pca_3,V1_pca_4,V12_pca_0,V12_pca_1,V12_pca_2,V12_pca_3,V12_pca_4,V35_pca_0,V35_pca_1,V35_pca_2,V35_pca_3,V35_pca_4,V53_pca_0,V53_pca_1,V53_pca_2,V53_pca_3,V53_pca_4,V75_pca_0,V75_pca_1,V75_pca_2,V75_pca_3,V75_pca_4,V95_pca_0,V95_pca_1,V95_pca_2,V95_pca_3,V95_pca_4,V138_pca_0,V138_pca_1,V138_pca_2,V138_pca_3,V138_pca_4,V167_pca_0,V167_pca_1,V167_pca_2,V167_pca_3,V167_pca_4,V217_pca_0,V217_pca_1,V217_pca_2,V217_pca_3,V217_pca_4,V279_pca_0,V279_pca_1,V279_pca_2,V279_pca_3,V279_pca_4,V322_pca_0,V322_pca_1,V322_pca_2,V322_pca_3,V322_pca_4,id_33_0,id_33_1,id_30_device,id_30_version,lastest_browser,cents,uid0,uid1,uid2,uid3,uid4,uid5,uid6,uid7,uid8,uid9,uid10,uid11,uid12,uid13,uid14,uid15,uid16,uid17,uid18,uid19,uid20,uid21,uid22,uid23,uid24,uid25,uid26,uid27,uid28,uid29,uid30,uid31,uid32,uid33,uid34,uid35,uid36,uid37,uid38,uid39,uid40,uid41,uid42,uid43,uid44,uid45,uid46,uid47,uid48,uid49,uid50,uid51,uid52,uid53,uid54,uid55,uid56,uid57,uid58,uid59,uid60,uid61,uid62,uid63,uid64,uid65,uid66,uid67,uid68,uid69,uid70,uid71,uid72,uid73,uid74,uid75,uid76,uid77,uid78,uid79,uid80,uid81,uid82,uid83,uid84,uid85,uid86,uid87,uid88,uid89,uid90,uid91,uid92,uid93,uid94,uid95,uid96,uid97,uid98,uid99,uid100,uid101,uid102,uid103,uid104,uid105,uid106,uid107,uid108,card1_TransactionAmt_mean,card1_TransactionAmt_std,card2_TransactionAmt_mean,card2_TransactionAmt_std,card3_TransactionAmt_mean,card3_TransactionAmt_std,card5_TransactionAmt_mean,card5_TransactionAmt_std,uid0_TransactionAmt_mean,uid0_TransactionAmt_std,uid1_TransactionAmt_mean,uid1_TransactionAmt_std,uid2_TransactionAmt_mean,uid2_TransactionAmt_std,uid3_TransactionAmt_mean,uid3_TransactionAmt_std,uid4_TransactionAmt_mean,uid4_TransactionAmt_std,uid5_TransactionAmt_mean,uid5_TransactionAmt_std,uid6_TransactionAmt_mean,uid6_TransactionAmt_std,uid7_TransactionAmt_mean,uid7_TransactionAmt_std,uid8_TransactionAmt_mean,uid8_TransactionAmt_std,uid9_TransactionAmt_mean,uid9_TransactionAmt_std,uid10_TransactionAmt_mean,uid10_TransactionAmt_std,uid11_TransactionAmt_mean,uid11_TransactionAmt_std,uid12_TransactionAmt_mean,uid12_TransactionAmt_std,uid13_TransactionAmt_mean,uid13_TransactionAmt_std,uid14_TransactionAmt_mean,uid14_TransactionAmt_std,uid15_TransactionAmt_mean,uid15_TransactionAmt_std,uid16_TransactionAmt_mean,uid16_TransactionAmt_std,uid17_TransactionAmt_mean,uid17_TransactionAmt_std,uid18

In [9]:
########################### Model params
SEED = 42
lgb_params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'metric': 'auc',
    'n_jobs': -1,
    'learning_rate': 0.01,
    'num_leaves': 2**8,
    'max_depth': -1,
    'tree_learner': 'serial',
    'colsample_bytree': 0.5,
    'subsample_freq': 1,
    'subsample': 0.7,
    'n_estimators': 3000,
    'max_bin': 255,
    'verbose': -1,
    'seed': SEED,
    'early_stopping_rounds': 100,
}

In [10]:
manual_rm_col = [
    'id_27', 'id_22', 'id_25', 'id_12', 'id_23', 'id_24', 'id_28', 'id_29',
    'id_35', 'M1', 'addr2_fq_enc', 'addr2','card3_TransactionAmt_std','id_26',
    'id_16', 'id_21','id_38', 'id_04', 'id_34'
]
rm_features = rm_features + manual_rm_col

In [11]:
features_columns = list(train)

for col in rm_features:
    if col in features_columns:
        features_columns.remove(col)
        
print('feature # used:', len(features_columns))

feature # used: 652


In [15]:
%%time 
test_predictions, auc_score = make_predictions(train, test, train_y, features_columns, lgb_params)
# baseline: 0.92028

Fold: 0  -  453219 137321
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.954402	valid_1's auc: 0.880714
[200]	training's auc: 0.97768	valid_1's auc: 0.892444
[300]	training's auc: 0.990765	valid_1's auc: 0.899169
[400]	training's auc: 0.996485	valid_1's auc: 0.903895
[500]	training's auc: 0.998587	valid_1's auc: 0.90615
[600]	training's auc: 0.999373	valid_1's auc: 0.907874
[700]	training's auc: 0.999718	valid_1's auc: 0.909074
[800]	training's auc: 0.999874	valid_1's auc: 0.910419
[900]	training's auc: 0.999947	valid_1's auc: 0.912042
[1000]	training's auc: 0.99998	valid_1's auc: 0.912745
[1100]	training's auc: 0.999993	valid_1's auc: 0.913413
[1200]	training's auc: 0.999997	valid_1's auc: 0.913998
[1300]	training's auc: 0.999999	valid_1's auc: 0.914575
[1400]	training's auc: 1	valid_1's auc: 0.914578
[1500]	training's auc: 1	valid_1's auc: 0.91477
[1600]	training's auc: 1	valid_1's auc: 0.915306
Early stopping, best iteration is:
[1582]	trainin

Fold: 1  -  488908 101632
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.956495	valid_1's auc: 0.913308
[200]	training's auc: 0.979091	valid_1's auc: 0.924513
[300]	training's auc: 0.991217	valid_1's auc: 0.931531
[400]	training's auc: 0.996632	valid_1's auc: 0.935786
[500]	training's auc: 0.998704	valid_1's auc: 0.939071
[600]	training's auc: 0.999436	valid_1's auc: 0.94107
[700]	training's auc: 0.999744	valid_1's auc: 0.942377
[800]	training's auc: 0.999884	valid_1's auc: 0.942999
[900]	training's auc: 0.99995	valid_1's auc: 0.94365
[1000]	training's auc: 0.99998	valid_1's auc: 0.944068
[1100]	training's auc: 0.999993	valid_1's auc: 0.944208
[1200]	training's auc: 0.999998	valid_1's auc: 0.94439
[1300]	training's auc: 0.999999	valid_1's auc: 0.944664
[1400]	training's auc: 1	valid_1's auc: 0.944872
[1500]	training's auc: 1	valid_1's auc: 0.945138
[1600]	training's auc: 1	valid_1's auc: 0.945229
[1700]	training's auc: 1	valid_1's auc: 0.945339
[

Fold: 2  -  497955 92585
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.951005	valid_1's auc: 0.907847
[200]	training's auc: 0.975636	valid_1's auc: 0.924667
[300]	training's auc: 0.989512	valid_1's auc: 0.934303
[400]	training's auc: 0.996141	valid_1's auc: 0.940224
[500]	training's auc: 0.998466	valid_1's auc: 0.943727
[600]	training's auc: 0.999327	valid_1's auc: 0.945736
[700]	training's auc: 0.999695	valid_1's auc: 0.947288
[800]	training's auc: 0.999859	valid_1's auc: 0.94819
[900]	training's auc: 0.999938	valid_1's auc: 0.948671
[1000]	training's auc: 0.999975	valid_1's auc: 0.949146
[1100]	training's auc: 0.999991	valid_1's auc: 0.949488
[1200]	training's auc: 0.999997	valid_1's auc: 0.949677
[1300]	training's auc: 0.999999	valid_1's auc: 0.949714
[1400]	training's auc: 1	valid_1's auc: 0.949984
[1500]	training's auc: 1	valid_1's auc: 0.950252
Early stopping, best iteration is:
[1482]	training's auc: 1	valid_1's auc: 0.950286
     Value  

Fold: 3  -  501214 89326
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.954088	valid_1's auc: 0.904574
[200]	training's auc: 0.977013	valid_1's auc: 0.917734
[300]	training's auc: 0.98962	valid_1's auc: 0.926099
[400]	training's auc: 0.995867	valid_1's auc: 0.930897
[500]	training's auc: 0.998412	valid_1's auc: 0.934168
[600]	training's auc: 0.999303	valid_1's auc: 0.935894
[700]	training's auc: 0.999674	valid_1's auc: 0.937106
[800]	training's auc: 0.999844	valid_1's auc: 0.937918
[900]	training's auc: 0.999928	valid_1's auc: 0.938674
[1000]	training's auc: 0.999969	valid_1's auc: 0.939423
[1100]	training's auc: 0.999987	valid_1's auc: 0.940101
[1200]	training's auc: 0.999995	valid_1's auc: 0.940505
[1300]	training's auc: 0.999998	valid_1's auc: 0.940601
[1400]	training's auc: 0.999999	valid_1's auc: 0.940926
[1500]	training's auc: 0.999999	valid_1's auc: 0.941068
[1600]	training's auc: 0.999999	valid_1's auc: 0.941069
[1700]	training's auc: 0.9

Fold: 4  -  504519 86021
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.952548	valid_1's auc: 0.924477
[200]	training's auc: 0.976539	valid_1's auc: 0.937251
[300]	training's auc: 0.989892	valid_1's auc: 0.942944
[400]	training's auc: 0.995801	valid_1's auc: 0.94657
[500]	training's auc: 0.998315	valid_1's auc: 0.948928
[600]	training's auc: 0.999242	valid_1's auc: 0.950325
[700]	training's auc: 0.999646	valid_1's auc: 0.951122
[800]	training's auc: 0.999837	valid_1's auc: 0.951641
[900]	training's auc: 0.999926	valid_1's auc: 0.952076
[1000]	training's auc: 0.999969	valid_1's auc: 0.952444
[1100]	training's auc: 0.999989	valid_1's auc: 0.952561
[1200]	training's auc: 0.999996	valid_1's auc: 0.952828
[1300]	training's auc: 0.999999	valid_1's auc: 0.953069
[1400]	training's auc: 1	valid_1's auc: 0.95309
Early stopping, best iteration is:
[1343]	training's auc: 0.999999	valid_1's auc: 0.953222
     Value                          Feature
0        0 

Fold: 5  -  506885 83655
Training until validation scores don't improve for 100 rounds.
[100]	training's auc: 0.95089	valid_1's auc: 0.910829
[200]	training's auc: 0.975633	valid_1's auc: 0.929319
[300]	training's auc: 0.989347	valid_1's auc: 0.939145
[400]	training's auc: 0.995785	valid_1's auc: 0.944332
[500]	training's auc: 0.998253	valid_1's auc: 0.948145
[600]	training's auc: 0.999237	valid_1's auc: 0.950298
[700]	training's auc: 0.999634	valid_1's auc: 0.951771
[800]	training's auc: 0.999822	valid_1's auc: 0.952812
[900]	training's auc: 0.999917	valid_1's auc: 0.95369
[1000]	training's auc: 0.999964	valid_1's auc: 0.954294
[1100]	training's auc: 0.999985	valid_1's auc: 0.954855
[1200]	training's auc: 0.999994	valid_1's auc: 0.955064
[1300]	training's auc: 0.999998	valid_1's auc: 0.955381
[1400]	training's auc: 0.999999	valid_1's auc: 0.955597
[1500]	training's auc: 1	valid_1's auc: 0.955729
[1600]	training's auc: 1	valid_1's auc: 0.95576
[1700]	training's auc: 1	valid_1's auc: 0.

CPU times: user 22h 9min 59s, sys: 4min 18s, total: 22h 14min 18s
Wall time: 1h 24min 44s


In [16]:
test_predictions.to_csv('../submissions/sub.csv', index=True) # LB 0.9495

In [14]:
# def make_test_predictions(tr_df, tt_df, y, features_columns, lgb_params, NFOLDS=2):

#     folds = KFold(n_splits=NFOLDS, shuffle=False, random_state=SEED)

#     X = tr_df[features_columns]
#     X_test = tt_df[features_columns]

#     pred_df = pd.DataFrame()
#     pred_df['TransactionID'] = tt_df.reset_index()['TransactionID']
#     pred_df['isFraud'] = np.zeros(len(pred_df))

#     predictions = np.zeros(len(tt_df))
#     oof = np.zeros(len(tr_df))

#     for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y, groups=split_groups)):
#         tr_x, tr_y = X.iloc[trn_idx, :], y.iloc[trn_idx]
#         vl_x, vl_y = X.iloc[val_idx, :], y.iloc[val_idx]

#         print('Fold:', fold_, ' - ', len(tr_x), len(vl_x))
#         tr_data = lgb.Dataset(tr_x, label=tr_y)
#         vl_data = lgb.Dataset(vl_x, label=vl_y)

#         estimator = lgb.train(
#             lgb_params,
#             tr_data,
#             valid_sets=[tr_data, vl_data],
#             verbose_eval=100,
#         )
        
#         pp_p = estimator.predict(X_test)
#         predictions += pp_p / NFOLDS

#         oof_preds = estimator.predict(vl_x)
#         oof[val_idx] = (oof_preds - oof_preds.min()) / (oof_preds.max() -
#                                                         oof_preds.min())

#         feature_imp = pd.DataFrame(sorted(zip(estimator.feature_importance(),X.columns)), columns=['Value','Feature'])
#         print(feature_imp)

#         del tr_x, tr_y, vl_x, vl_y, tr_data, vl_data
#         gc.collect()

#     pred_df['isFraud'] = predictions
#     pred_df = pred_df.set_index('TransactionID')

#     print('---------------------------------------')
#     print('OOF AUC:', metrics.roc_auc_score(y, oof))

#     return pred_df, metrics.roc_auc_score(y, oof)